In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/jane-street-market-prediction/example_sample_submission.csv
/kaggle/input/jane-street-market-prediction/features.csv
/kaggle/input/jane-street-market-prediction/example_test.csv
/kaggle/input/jane-street-market-prediction/train.csv
/kaggle/input/jane-street-market-prediction/janestreet/competition.cpython-37m-x86_64-linux-gnu.so
/kaggle/input/jane-street-market-prediction/janestreet/__init__.py


In [2]:
train = pd.read_csv('../input/jane-street-market-prediction/train.csv')
train = train.query('date > 85').reset_index(drop = True) 
train = train[train['weight'] != 0]

In [3]:
fill_mode = lambda col: col.fillna(col.mode()[0])
train = train.apply(fill_mode, axis=0)
train.isna().sum()

date           0
weight         0
resp_1         0
resp_2         0
resp_3         0
              ..
feature_126    0
feature_127    0
feature_128    0
feature_129    0
ts_id          0
Length: 138, dtype: int64

In [4]:
train['action'] = ((train['weight'].values * \
                         (train['resp_1'].values + \
                          train['resp_2'].values + train['resp_3'].values + train['resp_4'].values + train['resp'].values))/5 > 0).astype('int')

In [5]:
train = train.drop(['resp', 'resp_1', 'resp_2', 'resp_3', 'resp_4'], axis=1)

In [6]:
X = train.drop(['date', 'ts_id', 'action'], axis=1)
y = train['action']

In [7]:
from sklearn.preprocessing import QuantileTransformer

quant_transf = QuantileTransformer(n_quantiles=100, output_distribution='normal')
X_norm = quant_transf.fit_transform(X)

In [34]:
from sklearn.decomposition import PCA

pca = PCA(n_components=60)
pca.fit(X_norm)

PCA(n_components=60)

In [35]:
X_pca = pca.transform(X_norm)
df_X_pca = pd.DataFrame(X_pca)

In [36]:
df_X_pca

,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
0,10.267772,16.946523,-1.331063,-5.670721,-4.093065,2.104919,0.566892,-4.250220,-1.921255,0.970606,...,0.366612,0.031499,-0.688510,0.254045,0.350635,-0.021549,-0.095333,-0.499643,-0.262423,0.137681
1,-6.347576,13.478861,6.620328,-3.594931,-5.978475,3.056489,-0.094001,-3.234973,-2.120153,-1.623328,...,-0.062853,-0.215289,-1.132494,0.914764,0.338504,-0.019463,0.682655,0.950265,0.047911,-0.083753
2,8.932523,18.439425,0.463090,-1.298187,-2.387967,1.721530,1.101011,-3.905756,-1.883096,0.154013,...,-0.216718,-0.394815,-0.429627,-0.264379,0.226834,0.074856,-0.164028,-0.516284,-0.233366,0.135736
3,-4.202906,12.185362,8.171649,-3.729960,-6.939321,3.266822,3.768947,-2.715431,-3.053356,0.807371,...,0.282798,-0.023699,-1.064348,0.162919,-0.303998,-0.043396,0.524311,0.960083,-0.227284,0.119126
4,-4.196369,10.527558,9.632615,-5.211299,-6.616397,3.559874,1.816287,-2.397273,-2.785546,-0.278132,...,-0.452119,0.169474,-0.426684,0.320497,-0.495610,-0.127310,0.670623,1.439441,0.135713,-0.053754
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1571410,-7.383487,-8.093775,1.863693,-1.093240,-2.657842,3.243292,-2.186097,3.098878,0.466389,2.547661,...,0.506802,0.187615,0.048508,0.136285,0.628497,0.230144,-0.389141,0.279421,0.305769,0.233719
1571411,11.933524,-5.057887,-3.671960,-0.338875,4.053339,-1.803195,-0.185232,4.254357,-1.934252,1.895291,...,-0.718351,-1.359395,0.409063,0.126245,0.251997,-0.120654,0.155802,0.284568,0.034169,-0.161152
1571412,-4.419997,-4.594033,1.650150,-0.727765,-0.990756,3.741446,4.614244,1.951358,-1.840219,2.199059,...,-0.065476,0.542272,0.255529,0.299357,-0.388853,0.255215,-0.408657,-0.312628,0.048616,0.216179
1571413,-10.159801,-1.499152,-3.187130,1.567313,0.025382,-0.223769,1.413488,3.043434,0.410100,0.457263,...,1.040076,-0.318974,-1.103048,0.101136,0.684958,0.126496,-0.608576,0.106263,0.116390,0.428562


In [37]:
# check the variance in each component

y_variance = pca.explained_variance_ratio_
print(y_variance)
variance_sum = np.sum(y_variance)
variance_sum

[0.35015802 0.11083169 0.08052959 0.07169189 0.04331699 0.03749399
 0.0298198  0.02450167 0.01958024 0.018858   0.01673821 0.01480896
 0.01434448 0.01305356 0.01169462 0.00972094 0.00941423 0.00925625
 0.00889918 0.00767614 0.0069492  0.0069038  0.00672059 0.00587844
 0.00556627 0.00468511 0.00450775 0.00427327 0.00382346 0.00357803
 0.00345269 0.00259438 0.00232548 0.00228062 0.00225987 0.00220166
 0.00207961 0.00205003 0.00174569 0.00149185 0.00134726 0.00128668
 0.00118897 0.00118005 0.00116698 0.0010843  0.00099359 0.00092172
 0.00090574 0.00085781 0.00075814 0.00072046 0.00065098 0.00062772
 0.00055402 0.00053034 0.00046657 0.00046038 0.00038923 0.00038845]


0.99423566734869